### import

In [69]:
import pandas as pd
import numpy as np
import re, os
from os.path import join
from tqdm import tqdm
from file_function import read_filelist
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl import Workbook

# dtype 정의
debt_dtype = {'채무자키':str, '타채무자키':str, '담당자키':str}


### 파일명 바꾸기
- 채무조정안

In [ ]:
# 고객식별번호로 저장된 스캔파일 읽기
################################################
path_pdf = r"D:\3.자산\신용회복\솔림\1.신용회복\2.스캔파일\새 폴더"
docu = "채무조정안" # 신청인현황, 채무조정안, 원상회복내역
기준일 = "231004" # - 엑셀파일에 실효일자이 없을 때는 지정
# 업무파일/회신파일에서 다음을 칼럼으로 - 채무자키,고객명,고객식별번호,기준일    //////
ref_df = pd.read_excel(r"c://Users/SL/Desktop/통합 문서1.xlsx", dtype=str) 
################################################
filelist = read_filelist(path_pdf)

# 채무자키 정렬
ref_df.sort_values(['채무자키'], inplace=True)

# # 고객식별번호 중복제거 - 대표 채무자키에만 등록할 때 : 어차피 스캔파일을 순회하기 때문에 대표채무자키에만 등록되게 된다.
# ref_df.drop_duplicates(subset=['고객식별번호'], inplace=True, ignore_index=True)

In [ ]:
# 파일명 변경
for f in filelist : 
    stem, ext = os.path.splitext(f)
    temp = ref_df.query("고객식별번호==@stem")
    if "기준일" in temp.columns : 
        new_name = temp.채무자키.values[0] + "_" + temp.고객명.values[0] + "_" + docu +"_" + temp.기준일.values[0] + ext
    else : 
        new_name = temp.채무자키.values[0] + "_" + temp.고객명.values[0] + "_" + docu +"_" + 기준일 + ext
    os.rename(join(path_pdf, f), join(path_pdf, new_name))

### 담당자 변경

##### 기본변수 및 담당자로그파일 읽기

In [59]:
# 추심팀 명단 : 부서 == 추심 (다운후 유저구분까지만 남겨두자)       #######변경있는 경우#######
전사원명단 = pd.read_excel(join(r"D:\3.자산\기타 업무\담당자변경","0.사원_20231018_1552.xlsx"), dtype=str)
사원_지점 = 전사원명단.query('부서=="추심"')[["사원성명", "지점"]].copy()
추심팀명단 = 사원_지점.사원성명.tolist()
print(추심팀명단)

# 주간 담당자로그 파일 읽기
###################################
wd = r"D:\3.자산\기타 업무\담당자변경"
filename_log = "담당자로그_202309.xlsx"
기간 = "2309"
###################################
log = pd.read_excel(join(wd,filename_log), dtype=debt_dtype)

# 채무자키로 그룹화 : 변경일순으로 정렬되어 있으므로 순회하면 된다.
log_grouped = log.groupby('채무자키')
log_grouped.get_group(list(log_grouped.groups.keys())[0])

['김대홍', '신경용', '김용기', '김천곤', '박병원', '서삼석', '안병렬', '양미정', '엄주억', '오남진', '지영숙', '허곤', '김지홍', '정재하', '확인용']


c:\Users\sl\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,채무자키,채무자,변경전,변경후,담당변경사유,요청자,변경일,변경자
25391,20407805,문영민,전나영,신용회복,신용회복중,전나영,2023-09-15 17:31:39,전나영


##### 데이터 조작

In [76]:
# 제외건, 배정건 만들기

# 제외건
제외_채무자키_list = []
제외_채무자명_list = []
제외_변경전_list = []
제외_변경후_list = []
제외_최종변경일_list = []

# 배정건
배정_채무자키_list = []
배정_채무자명_list = []
배정_변경전_list = []
배정_변경후_list = []
배정_최종변경일_list = []


for k, v in log_grouped :
    변경전추심팀 = list(x for x in v.변경전.values if x in 추심팀)
    변경전관리팀 = list(x for x in v.변경전.values if x not in 추심팀)
    최종담당자 = v.변경후.values[-1]

    # 제외건에 해당하는 조건
    if 변경전추심팀 and  최종담당자 not in 추심팀 : 
        제외_채무자키_list.append(k) # k는 str임
        제외_채무자명_list.append(v.채무자.values[0]) # 똑같으므로
        제외_변경전_list.append(변경전추심팀[-1]) # 여러명인 경우, 마지막 담당자
        제외_변경후_list.append(최종담당자)
        제외_최종변경일_list.append(v.변경일.values[-1][:10])
    
    # 배정건에 해당하는 조건
    if 변경전관리팀 and  최종담당자 in 추심팀 : 
        배정_채무자키_list.append(k) # k는 str임
        배정_채무자명_list.append(v.채무자.values[0]) # 똑같으므로
        배정_변경전_list.append(">".join(변경전관리팀)) # 진행상황 알수 있도록 > 로 연결
        배정_변경후_list.append(최종담당자)
        배정_최종변경일_list.append(v.변경일.values[-1][:10])

# df만들기
columns = ["채무자키", "채무자명", "변경전", "변경후", "최종변경일"]
제외_data = {col: values for col, values in zip(columns, [제외_채무자키_list, 제외_채무자명_list, 제외_변경전_list, 제외_변경후_list, 제외_최종변경일_list])}
제외건 = pd.DataFrame(제외_data)

배정_data = {col: values for col, values in zip(columns, [배정_채무자키_list, 배정_채무자명_list, 배정_변경전_list, 배정_변경후_list, 배정_최종변경일_list])}
배정건 = pd.DataFrame(배정_data)

# 지점열 추가
제외건 = pd.merge(제외건,사원_지점, left_on='변경전', right_on='사원성명', how='left')
제외건.drop('사원성명', axis=1, inplace=True)

배정건 = pd.merge(배정건,사원_지점, left_on='변경후', right_on='사원성명', how='left')
배정건.drop('사원성명', axis=1, inplace=True)

# 지점-담당자 이름순 정렬
제외건.sort_values(['지점', '변경전'], ascending=True, inplace=True)
배정건.sort_values(['지점', '변경후'], ascending=True, inplace=True)

print("제외건 개수 : ", len(제외건))
print("배정건 개수 : ", len(배정건))

제외건 개수 :  100
배정건 개수 :  1


##### 엑셀 만들기

In [79]:
# 엑셀 쓰기

# 시트 만들기
wb = Workbook()
ws1 = wb.active
ws1.title = "제외건"
ws2 = wb.create_sheet("배정건")

# 시트에 데이터 입력
for r in dataframe_to_rows(제외건, index=False, header=True) : ws1.append(r)
for r in dataframe_to_rows(배정건, index=False, header=True) : ws2.append(r)

# 스타일 지정
# 폰트
font_data = Font(name='NN30', size=10)
font_col = Font(name='NN30', size=10, color='FFFFFF', bold=True)
# 컬럼 배경색
fill_col = PatternFill(fill_type='solid', start_color='FF0072C6', end_color='FF0072C6') # 칼럼명
fill_highlight = PatternFill(fill_type='solid', start_color='F79646', end_color='F79646') # 칼럼명
# 테두리 : 직선
border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
# 정렬
alignment = Alignment(horizontal='center', vertical='center')

# 스타일 적용
for ws in [ws1, ws2] : 

    # 시트 첫행(칼럼명)
    for cell in ws[1] :
        if (cell.value =="변경전" and ws.title=="제외건") or (cell.value =="변경후" and ws.title=="배정건") :
            cell.fill = fill_highlight

        else :     
            cell.fill = fill_col
        cell.font = font_col
        cell.border = border
        cell.alignment = alignment
    
    # 2행부터 순회(데이터)
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row) : 
        for cell in row :
            cell.font = font_data
            cell.border = border
            cell.alignment = alignment

# 파일 저장
wb.save(join(wd, "담당자변경_"+기간+".xlsx"))